In [1]:
import numpy as np
import os


In [2]:
from action import c3d

In [3]:
help(c3d)

Help on package action.c3d in action:

NAME
    action.c3d

PACKAGE CONTENTS
    callbacks
    models
    train_utils
    utils

FILE
    /home/akashdeepjassal/Documents/package/lib/python3.6/site-packages/action_recognition_keras-0.1-py3.6.egg/action/c3d/__init__.py




In [4]:
from action.c3d import utils

In [5]:
# help(utils)
utils.download('UCF-101')

In [6]:
utils.unrar('UCF101.rar','dataset')
utils.unzip('UCF101_Action_detection_splits.rar','dataset')

In [ ]:
from action.c3d.models import c3d_model
from keras.optimizers import SGD,Adam
from keras.utils import np_utils
# from schedules import onetenth_4_8_12
import numpy as np
import random
import cv2
import os
import random
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
from actions.c3d.train_utils import generator_train_batch, generator_val_batch, plot_history, save_history
from actions.c3d.callbacks import Step
from action.c3d.utils import vid2img, label2text

def onetenth_4_8_12(lr):
    steps = [4, 8,12]
    lrs = [lr, lr/10, lr/100,lr/1000]
    return Step(steps, lrs)
    

def main():
    img_path = 'ucfimgs/'
    train_file = 'train_list.txt'
    test_file = 'test_list.txt'
    label2text('action-recognition-keras/dataset/UCF-101/train_file.txt', 'action-recognition-keras/dataset/UCF-101/test_file.txt','dataset/UCF-101/video/')
    f1 = open(train_file, 'r')
    f2 = open(test_file, 'r')
    lines = f1.readlines()
    f1.close()
    train_samples = len(lines)
    lines = f2.readlines()
    f2.close()
    val_samples = len(lines)

    num_classes = 101
    batch_size = 16
    epochs = 16

    model = c3d_model()
    lr = 0.005
    sgd = SGD(lr=lr, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.summary()
    history = model.fit_generator(generator_train_batch(train_file, batch_size, num_classes,img_path),
                                  steps_per_epoch=train_samples // batch_size,
                                  epochs=epochs,
                                  callbacks=[onetenth_4_8_12(lr)],
                                  validation_data=generator_val_batch(test_file,
                                        batch_size,num_classes,img_path),
                                  validation_steps=val_samples // batch_size,
                                  verbose=1)
    if not os.path.exists('results/'):
        os.mkdir('results/')
    plot_history(history, 'results/')
    save_history(history, 'results/')
    model.save_weights('results/weights_c3d.h5')


if __name__ == '__main__':
    main()